In [48]:
import numpy as np        
from itertools import combinations, chain

In [49]:
def load_transactions(path_to_data, order):
    data_trancations = []
    with open(path_to_data, 'r') as file:
        for line in file:
            str_line = list(line.strip().split(', '))
            _t = list(np.unique(str_line))
            _t.sort(key = lambda x : order.index(x))
            data_trancations.append(_t)
    return data_trancations


In [50]:
file_path = 'apriori_itemset.txt'
order = ['I' + str(i) for i in range(1, 6)]

In [51]:
Transcations = load_transactions(file_path, order)
num_trans = len(Transcations)

In [52]:
c = {} # candidate set  uses l[k-1] 
l = {} # frequent set uses c[k]
supp_count_l = {}
itemset_size = 1
min_support = 2/9
min_confidence = 3/10 
discarded = {itemset_size : []}
c.update({itemset_size : [[i] for i in order]})

In [53]:
print(c)

{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}


In [54]:
def count_occurences(itemset, transcations):
    count = 0
    for i in range(len(transcations)):
        if set(itemset).issubset(transcations[i]):
            count+=1
    return count

In [55]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x : order.index(x))
    it2.sort(key = lambda x : order.index(x))
    
    for i in range(len(it1)-1):
        if it1[i] != it2[i]:
            return []
    x = it1[-1]
    y = it2[-1]
    if order.index(x) < order.index(y):
        return it1 + [it2[-1]]
    return []
def join_set_itemsets(set_of_items, order):
    c = []
    for i in range(len(set_of_items)):
        for j in range(i+1, len(set_of_items)):
            it_out = join_two_itemsets(set_of_items[i], set_of_items[j], order)
            
            if len(it_out) > 0:
                c.append(it_out)
    return c

In [56]:
def print_table(t, supp):
    print('Itemset | frequency')
    for k in range(len(t)):
        print(f"{t[k]} : {supp[k]}")
    print("\n\n")        

In [57]:
def get_frequent(itemsets, transcations, min_support, prev_discarded):
    L = []
    supp_count= []
    new_discarded = []
    num_trans = len(transcations)
    
    k = len(prev_discarded.keys()) #is prev_discarded an dict....???? --> Yeah with keys as [1, 2, 3...] and value of each key is the list of discarded elements in that iteration
    for s in range(len(itemsets)):
        discarded_previous = False
        if k>1:
            length_of_each_itemset = len(itemsets[0]) ### changes
            for it in prev_discarded[length_of_each_itemset-1]: ###changes
                if len(it) == 0:
                    pass
                else:
                    if set(it).issubset(set(itemsets[s])): #### problem is here...........
                        discarded_previous = True
                        break
        if not discarded_previous:
            count = count_occurences(itemsets[s], transcations)
            if count/num_trans >= min_support:
                L.append(itemsets[s])
                supp_count.append(count) ## what is the use of this.......
            else:
                new_discarded.append(itemsets[s])
    
    return L, supp_count, new_discarded

In [58]:
supp_count_l = {}
f, sup, new_discarded = get_frequent(c[itemset_size], Transcations, min_support, discarded)

In [59]:
l.update({itemset_size : f})
supp_count_l.update({itemset_size : sup})
discarded.update(new_discarded)
print(f)

[['I1'], ['I2'], ['I3'], ['I4'], ['I5']]


In [60]:
print_table(l[1], supp_count_l[1])

Itemset | frequency
['I1'] : 4
['I2'] : 4
['I3'] : 5
['I4'] : 3
['I5'] : 2





In [61]:
k = itemset_size 
converged = False
while not converged:
    k+=1
    c.update({k : join_set_itemsets(l[k-1], order)})
    print(f"c[{k}]")
    print_table(c[k], [count_occurences(i, Transcations) for i in c[k]])
    f, sup, new_discarded = get_frequent(c[k], Transcations, min_support, discarded)
    l.update({k : f})
    supp_count_l.update({k : sup})
    discarded.update({k : new_discarded})
    length = len(l[k])
    if length == 0:
        converged = True
    else:
        print(f'l[{k}]')
        print_table(l[k], supp_count_l[k])


c[2]
Itemset | frequency
['I1', 'I2'] : 3
['I1', 'I3'] : 3
['I1', 'I4'] : 0
['I1', 'I5'] : 0
['I2', 'I3'] : 2
['I2', 'I4'] : 1
['I2', 'I5'] : 1
['I3', 'I4'] : 2
['I3', 'I5'] : 1
['I4', 'I5'] : 2



l[2]
Itemset | frequency
['I1', 'I2'] : 3
['I1', 'I3'] : 3
['I2', 'I3'] : 2
['I3', 'I4'] : 2
['I4', 'I5'] : 2



c[3]
Itemset | frequency
['I1', 'I2', 'I3'] : 2



l[3]
Itemset | frequency
['I1', 'I2', 'I3'] : 2



c[4]
Itemset | frequency





In [62]:
def powerset(iterable):
    
    return list(chain.from_iterable(combinations(iterable, r) for r in range(1, len(iterable)+1)))

In [63]:
def write_rules(t, s_r, z, conf, sup_s_r, lift, num_trans):
    out_rule = ""
    out_rule += f"Frequent Itemsets --> {t}    items_buyed --> {z}    recommended_items--> {s_r}    confidence --> {conf}    support --> {supp_s_r}    lift --> {lift}    number_of_trancations --> {num_trans}"
    return out_rule

In [75]:
assoc_rules_str = ""
for i in range(1, len(l)):
    for j in range(len(l[i])):
        s = powerset(set(l[i][j]))  ## powerset is set of all subsets of the give set
        s.pop()
        for z in s:
            # if he buys z i want to recommend s_r
            z = set(z)
            t = set(l[i][j])
            s_r = set(t-z)
            sup_t = count_occurences(t, Transcations)
            supp_s_r = count_occurences(s_r, Transcations)
            conf = sup_t / supp_s_r
            lift = conf / (supp_s_r / num_trans)
            if conf >= min_confidence and supp_s_r >= min_support:
                assoc_rules_str += write_rules(t, s_r, z, conf, supp_s_r, lift, num_trans)
                assoc_rules_str +='\n'
                assoc_rules_str +='\n'
            

## here it recommends a-->b and also b-->a since if one exisits the other will also exsist

In [74]:
print(assoc_rules_str)


Frequent Itemsets --> {'I2', 'I1'}    items_buyed --> {'I2'}    recommended_items--> {'I1'}    confidence --> 0.75    support --> 4    lift --> 1.3125    number_of_trancations --> 7

Frequent Itemsets --> {'I2', 'I1'}    items_buyed --> {'I1'}    recommended_items--> {'I2'}    confidence --> 0.75    support --> 4    lift --> 1.3125    number_of_trancations --> 7

Frequent Itemsets --> {'I3', 'I1'}    items_buyed --> {'I3'}    recommended_items--> {'I1'}    confidence --> 0.75    support --> 4    lift --> 1.3125    number_of_trancations --> 7

Frequent Itemsets --> {'I3', 'I1'}    items_buyed --> {'I1'}    recommended_items--> {'I3'}    confidence --> 0.6    support --> 5    lift --> 0.84    number_of_trancations --> 7

Frequent Itemsets --> {'I3', 'I2'}    items_buyed --> {'I3'}    recommended_items--> {'I2'}    confidence --> 0.5    support --> 4    lift --> 0.875    number_of_trancations --> 7

Frequent Itemsets --> {'I3', 'I2'}    items_buyed --> {'I2'}    recommended_items--> {'I3'